# Introducing HOOMD-blue

This hands-on workshop introduces the basic concepts of HOOMD-blue and will teach you how to run a hard particle Monte Carlo simulation. For more details, information that may be helpful when completing the exercises, see the [HOOMD-blue documentation](https://hoomd-blue.readthedocs.io/).

**FIRST:** Click the "Copy to Drive" button in the toolbar to save this notebook and any edits that you make to your Google Drive.

## Utility code

Please ignore the content of these cells, they provide some helpful methods but are not part of the workshop.

Run them one at a time, then click the arrow by "Utility code" to hide them.

In [ ]:
# This cell installs HOOMD-blue in Google Colab. You can delete it if you run locally.
try:
    import google

    !pip install -q condacolab
    import condacolab

    condacolab.install_from_url(
        'https://github.com/glotzerlab/hoomd-workshop/releases/download/2022.0.1/hoomd-workshop-2022.0.1-Linux-x86_64.sh'
    )
except ModuleNotFoundError:
    pass

In [ ]:
# Please ignore the content of this cell. Run it after condacolab finishes
# and proceed to the workshop content below.

# This cell defines a `render` function that code below uses to visualize system
# configurations.
import math
import warnings

import fresnel
import IPython
import packaging.version

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=400, h=400)

FRESNEL_MIN_VERSION = packaging.version.parse("0.13.0")
FRESNEL_MAX_VERSION = packaging.version.parse("0.14.0")


def render(snapshot):
    if ('version' not in dir(fresnel) or packaging.version.parse(
            fresnel.version.version) < FRESNEL_MIN_VERSION
            or packaging.version.parse(
                fresnel.version.version) >= FRESNEL_MAX_VERSION):
        warnings.warn(f"Unsupported fresnel version {fresnel.version.version}"
                      " - expect errors.")
    L = snapshot.configuration.box[0]
    vertices = [
        (-0.5, 0, 0),
        (0.5, 0, 0),
        (0, -0.5, 0),
        (0, 0.5, 0),
        (0, 0, -0.5),
        (0, 0, 0.5),
    ]
    poly_info = fresnel.util.convex_polyhedron_from_vertices(vertices)

    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.ConvexPolyhedron(scene,
                                                 poly_info,
                                                 N=snapshot.particles.N)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear(
        [0.01, 0.74, 0.26]),
                                                  roughness=0.5)
    geometry.position[:] = snapshot.particles.position[:]
    if snapshot.particles.orientation is not None:
        geometry.orientation[:] = snapshot.particles.orientation[:]
    geometry.outline_width = 0.01
    box = fresnel.geometry.Box(  # noqa: F841  (box is used)
        scene,
        snapshot.configuration.box,
        box_radius=.02)

    scene.lights = [
        fresnel.light.Light(direction=(0, 0, 1),
                            color=(0.5, 0.5, 0.5),
                            theta=math.pi),
        fresnel.light.Light(direction=(1, 1, 1),
                            color=(1.1, 1.1, 1.1),
                            theta=math.pi / 3)
    ]
    scene.camera = fresnel.camera.Orthographic(position=(L * 2, L, L * 2),
                                               look_at=(0, 0, 0),
                                               up=(0, 1, 0),
                                               height=L * 1.4 + 1)
    scene.background_color = (1, 1, 1)
    return IPython.display.Image(tracer.sample(scene, samples=20)._repr_png_())

## Core objects

Import the HOOMD-blue Python package:

In [ ]:
import hoomd

### Selecting a device

Choose the hardware device to use:

In [ ]:
# gpu = hoomd.device.GPU()

Try creating a CPU device:

In [ ]:
# Complete the code.
# cpu =
cpu = hoomd.device.CPU()  # Solution.

### Creating a Simulation

Now, instantiate a **Simulation** with the chosen device.

In [ ]:
sim = hoomd.Simulation(device=cpu, seed=1)

A newly constructed **Simulation** has no **state**:

In [ ]:
print(sim.state)

### Populating the simulation state

Use [`freud`](https://freud.readthedocs.io) to generate points on a lattice for the initial condition and place them in a **Snapshot**:

In [ ]:
import freud
import gsd.hoomd

In [ ]:
snapshot = gsd.hoomd.Snapshot()
box, positions = freud.data.UnitCell.fcc().generate_system(num_replicas=3,
                                                           scale=1.5)
snapshot.particles.N = len(positions)
snapshot.particles.position = positions
snapshot.particles.types = ['octahedron']
snapshot.configuration.box = [box.Lx, box.Ly, box.Lz, 0, 0, 0]
render(snapshot)

Initialize the simulation **state** from the snapshot:

In [ ]:
sim.create_state_from_snapshot(snapshot)

Use `gsd` to write the snapshot out to a file (optional):

In [ ]:
with gsd.hoomd.open(name='octahedra.gsd', mode='wb') as f:
    f.append(snapshot)

Try initializing a new simulation `sim2` from the file `octahedra.gsd`:

In [ ]:
# Complete the code.
# sim2 = hoomd.Simulation(
# sim2.

sim2 = hoomd.Simulation(device=cpu)  # Solution.
sim2.create_state_from_gsd('octahedra.gsd')  # Solution.

### Adding the integrator and other operations

The **integrator** determines what kind of simulation HOOMD-blue executes. Here, we will perform hard particle Monte Carlo (HPMC) simulations of octahedra.

The **ConvexPolyhedron** **integrator** implements HPMC simulations - Create one:

In [ ]:
mc = hoomd.hpmc.integrate.ConvexPolyhedron()

Set the `shape` *property* to define the **particle shape**.
A convex polyhedron is defined by the convex hull of a [set of vertices](https://en.wikipedia.org/wiki/Octahedron):

In [ ]:
mc.shape['octahedron'] = dict(vertices=[
    (-0.5, 0, 0),
    (0.5, 0, 0),
    (0, -0.5, 0),
    (0, 0.5, 0),
    (0, 0, -0.5),
    (0, 0, 0.5),
])

Set the maximum trial displacement `d` and rotation `a`:

In [ ]:
mc.d['octahedron'] = 0.15
mc.a['octahedron'] = 0.2

Add the HPMC **integrator** to the **Simulation** operations:

In [ ]:
sim.operations += mc

## Running the simulation

Use the GSD writer to save simulation snapshots to a trajectory file:

In [ ]:
gsd_writer = hoomd.write.GSD(filename='trajectory.gsd',
                             trigger=hoomd.trigger.Periodic(1000),
                             mode='wb')

Add `gsd_writer` to the simulation's operations:

In [ ]:
# Add code.

sim.operations += gsd_writer  # Solution.

Run the simulation for 10,000 steps:

In [ ]:
# Add code.
sim.run(10000)  # Solution.

Look at the final state of the system:

In [ ]:
with gsd.hoomd.open('trajectory.gsd') as trajectory:
    image = render(trajectory[-1])
image

Check the translation trial move acceptance:

In [ ]:
accepted_moves, rejected_moves = mc.translate_moves
accepted_moves / (accepted_moves + rejected_moves)

What is the rotation trial move acceptance?

In [ ]:
# Add code.
accepted_moves, rejected_moves = mc.rotate_moves  # Solution.
accepted_moves / (accepted_moves + rejected_moves)  # Solution.

Add a **Tuner** that adjusts `a` and `d` to achieve a target acceptance ratio of 0.2:

In [ ]:
move_size_trigger = hoomd.trigger.Periodic(10)
tune_move_size = hoomd.hpmc.tune.MoveSize.scale_solver(
    moves=['a', 'd'],
    target=0.2,
    trigger=move_size_trigger,
    max_translation_move=1.0,
    max_rotation_move=0.5)
sim.operations += tune_move_size

Run the simulation for 1,000 steps:

In [ ]:
# Add code.
sim.run(1000)  # Solution.

Now the acceptance ratio is close to 0.2:

In [ ]:
accepted_moves, rejected_moves = mc.translate_moves
accepted_moves / (accepted_moves + rejected_moves)

`MoveSize` changed the maximum trial move sizes to achieve this:

In [ ]:
mc.d['octahedron']

What is the volume fraction of this simulation?

In [ ]:
V_particle = 1 / 3 * math.sqrt(2) * (math.sqrt(2) / 2)**3
initial_volume_fraction = (sim.state.N_particles * V_particle
                           / sim.state.box.volume)
print(initial_volume_fraction)

## Exercise: Compressing the system to a target volume fraction

Self-assembly occurs at higher volume fractions, typically above 0.5. As an exercise, add the code needed to compress the simulation to a target volume fraction of 0.5.

Step 1: Determine the simulation box at the target packing fraction:

In [ ]:
# Complete the code.
final_volume_fraction = 0.5
initial_box = sim.state.box
final_box = hoomd.Box.from_box(initial_box)
# final_box.volume =
final_box.volume = sim.state.N_particles * V_particle / final_volume_fraction  # Solution.

Step 2: Add an operation that compresses the system.

Tip: Look in the [HOOMD-blue documentation](https://hoomd-blue.readthedocs.io/)!

In [ ]:
# Add code.
compress = hoomd.hpmc.update.QuickCompress(trigger=hoomd.trigger.Periodic(10),
                                           target_box=final_box)  # Solution.
sim.operations += compress  # Solution.

Step 3: Run the simulation until the compression is *complete*.
Hint: Look at the documentation for the `complete` attribute.

In [ ]:
# Add code.
while not compress.complete:  # Solution.
    sim.run(100)  # Solution.

The volume fraction should now be 0.5:

In [ ]:
print(sim.state.N_particles * V_particle / sim.state.box.volume)

And the simulation should appear dense:

In [ ]:
render(sim.state.get_snapshot())

At home: Review the full version of this tutorial in the [HOOMD-blue documentation](https://hoomd-blue.readthedocs.io/) to learn how to equilibrate the system and analyze the output to find when it crystallizes.